In [1]:
import cv2
import numpy as np
import sys

import torch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data

import matplotlib.pyplot as plt

from torchviz import make_dot

sys.path.append('/home/meribejayson/Desktop/Projects/SharkCNN/training_models/dataloaders/')

megaset_train_images_path = "/home/meribejayson/Desktop/Projects/SharkCNN/datasets-reduced/megaset/train/images/"

from get_single_image_train_features import SharkImageFeatureGen as FeatureGenerator

In [2]:
# image_name = "dji206-1175.jpg"
# image_name = "dji343-7553.jpg"
image_name = "dji358-1099.jpg"

feature_gen = FeatureGenerator()
features = feature_gen.generate_image_features(image_name)

In [3]:
torch.manual_seed(12)

if not torch.cuda.is_available():
    raise Exception("Couldn't find CUDA")

device = torch.device("cuda")

In [4]:
class LogisticRegresion(nn.Module):

    def __init__(self, input_size):
        super().__init__()
        
        self.linear = nn.Linear(input_size,1)
        self.sig = nn.Sigmoid()

    def forward(self, x):
        x = self.linear(x)
        
        return self.sig(x)

In [5]:
class ANN(nn.Module):

    def __init__(self, input_size):
        super().__init__()
        
        hidden_layer_num = int(input_size / 2)

        self.linear1 = nn.Linear(input_size, hidden_layer_num)
        self.relu1 = nn.ReLU()
        self.linear2 = nn.Linear(hidden_layer_num + input_size, 1)
        self.sig = nn.Sigmoid()
        

    def forward(self, x):
        h = self.linear1(x)
        h = self.relu1(h)
        h = torch.cat((x, h), dim=1)
        h = self.linear2(h)
        
        return self.sig(h)

In [6]:
ann_state = torch.load("/home/meribejayson/Desktop/Projects/SharkCNN/training_models/ANN/train-4/ann_weights_train_4.tar")
lr_state = torch.load("/home/meribejayson/Desktop/Projects/SharkCNN/training_models/LOGISTIC-REG/train-final-6/lr_weights_train_6.tar")

ann_model = ANN(85)
ann_model.load_state_dict(ann_state)
ann_model.to(device)


lr_model = LogisticRegresion(85)
lr_model.load_state_dict(lr_state)
lr_model.to(device)

LogisticRegresion(
  (linear): Linear(in_features=85, out_features=1, bias=True)
  (sig): Sigmoid()
)

In [7]:
data_inputs = features[:, :-1].to(device).float()
data_labels = features[:, -1].to(device).float()

ann_preds = ann_model(data_inputs)
lr_preds = lr_model(data_inputs)

dot = make_dot(ann_preds, params=dict(ann_model.named_parameters()))
dot.render('./visual/ann_model_visualization', format='png')

dot = make_dot(lr_preds, params=dict(lr_model.named_parameters()))
dot.render('./visual/lr_model_visualization', format='png')

'visual/lr_model_visualization.png'

In [9]:
import torch
import torch.onnx

# Define or load your model
model = ANN(85)

# Load the model dictionary (Assuming model_dict is your model state dictionary)
model_dict_path = '/home/meribejayson/Desktop/Projects/SharkCNN/training_models/ANN/train-4/ann_weights_train_4.tar'
model.load_state_dict(torch.load(model_dict_path))

# Set the model to evaluation mode
model.eval()

# Create a dummy input that matches the input size the model expects
# Example for a model that expects input of size [batch_size, channels, height, width]
dummy_input = torch.randn(1, 85)  # Adjust the size as per your model's requirement

# Define the path for the ONNX model
onnx_model_path = 'ann.onnx'

# Export the model
torch.onnx.export(model,               # model being run
                  dummy_input,         # model input (or a tuple for multiple inputs)
                  onnx_model_path,     # where to save the model (can be a file or file-like object)
                  export_params=True,  # store the trained parameter weights inside the model file
                  opset_version=11,    # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names=['input'],   # the model's input names
                  output_names=['output'], # the model's output names
                  dynamic_axes={'input': {0: 'batch_size'},  # variable length axes
                                'output': {0: 'batch_size'}})

print(f'Model has been converted to ONNX format and saved to {onnx_model_path}')

Model has been converted to ONNX format and saved to ann.onnx
